<a href="https://colab.research.google.com/github/sayoojbk/TTWM/blob/master/Cat_dog_small_dataset_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following code mounts my Google drive to the Colab environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd My Drive

/content/drive/My Drive


In [4]:
cd Datasets

/content/drive/My Drive/Datasets


In [5]:
cd Cat-dog-classification

/content/drive/My Drive/Datasets/Cat-dog-classification


In [0]:
## Installing the Pytorch package to work with in Google Co

In [7]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 


    100% |████████████████████████████████| 592.3MB 49.4MB/s 


In [8]:
!pip3 install torchvision


    100% |████████████████████████████████| 61kB 4.2MB/s 
    100% |████████████████████████████████| 2.0MB 14.8MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
# importing libraries from the module

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import  Variable



In [0]:
data_transform = transforms.Compose([
        
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


In [0]:
train_data = datasets.ImageFolder("train",transform=data_transform)


In [0]:
num_epochs = 15
num_classes = 2
batch_size = 300
learning_rate = 0.05

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

In [0]:
# Create Model Class
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    # Convolution 1
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.relu1 = nn.ReLU()
    
    # Max pool 1
    self.maxpool1 = nn.AdaptiveMaxPool2d(output_size=14)
    
    # Convolution 2
    self.cnn2 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.relu2 = nn.ReLU()
    
    # Max pool 2
    self.maxpool2 = nn.AdaptiveMaxPool2d(output_size=7)
    
    # FC (readout)
    self.fc1 = nn.Linear(32*7*7, 4)
   
    
  def forward(self, x):
    # Convolution 1
    out = self.cnn1(x)
    out = self.relu1(out)
    
    # Max pool 1
    out = self.maxpool1(out)
    
    # Convolution 2
    out = self.cnn2(x)
    out = self.relu2(out)
    
    # Max pool 2
    out = self.maxpool2(out)
    
    # Resize
    # 100 is the batch size...
    # (100, 32, 7, 7) -> (100, 32*7*7)
    out = out.view(out.size(0), -1)
    
    # Linear (readout)
    out = self.fc1(out)
    
    return out

model = ConvNet().cuda()

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# Train the model
import time
tick = time.time()

iter = 0
for epoch in range(num_epochs):
  for i,(images, labels) in enumerate(train_loader):
    
    
    images = Variable(images.cuda()) # No need to resize
    labels = Variable(labels.cuda())
    
    optimizer.zero_grad()
    
    outputs = model(images)
    
    loss = criterion(outputs, labels)
    
    loss.backward()
    
    optimizer.step()

AttributeError: ignored